In [18]:
from pymilvus import connections, db, FieldSchema, CollectionSchema, DataType, Collection, utility

import pandas as pd

from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

In [4]:
conn = connections.connect(host="127.0.0.1", port=19530)

# database = db.create_database("movies_database")

In [7]:
db.using_database("movies_database")

In [22]:
# connections.disconnect(alias="default")

In [20]:
# db.drop_database("movies_database")

# db.list_database()

['default']

In [19]:
# Описание схемы для эмбеддингов фильмов
fields = [
    FieldSchema(name="movie_id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024)
]

# Создание схемы коллекции
schema = CollectionSchema(fields, description="Movie embeddings collection")

if "movies_collection" in utility.list_collections():
    Collection("movies_collection").drop()

# Создание коллекции
collection = Collection(name="movies_collection", schema=schema)

# Определение параметров индекса
index_params = {
    "metric_type": "IP",       # Тип метрики расстояния, например, L2 или IP
    "index_type": "IVF_FLAT",  # Тип индекса, например, IVF_FLAT
    "params": {"nlist": 512}   # Параметры индекса (nlist – это число кластеров)
}

# Создание индекса для поля embedding
collection.create_index(field_name="embedding", index_params=index_params)

# Убедитесь, что индекс создан
utility.index_building_progress("movies_collection")

# Загрузка коллекции в память (Milvus управляет хранением в памяти для быстрой обработки)
collection.load()

In [10]:
df_movies = pd.read_csv('../../data/merged_movies.csv')
df_movies_small = df_movies.iloc[:800]

In [20]:
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # для мака

# model_name = 'intfloat/multilingual-e5-large'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name).to(device)

# # Функция для генерации эмбеддингов
# def get_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
#     outputs = model(**inputs)
#     # Переносим тензор на CPU перед дальнейшими преобразованиями
#     embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
#     return embeddings[0]

# Загрузка модели и токенизатора
model_name = 'intfloat/multilingual-e5-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# # Функция для генерации эмбеддингов
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Используем среднее по последнему скрытому слою
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]



# Обработка каждого фильма для получения эмбеддинга
def generate_movie_embeddings(movie):
    # Конкатенация данных из нужных колонок
    text = f"{movie['title']} | {movie['genres']} | {movie['overview']} | {movie['production_countries']} | {movie['runtime']} | {movie['spoken_languages']} | {movie['vote_average']} | {movie['vote_count']}"
    return get_embedding(text)

In [21]:
print("Generating embeddings...")
tqdm.pandas()  # Инициализация прогресс-бара для pandas
df_movies_small['embedding'] = df_movies_small.progress_apply(generate_movie_embeddings, axis=1)

# Преобразование данных для вставки в Milvus
movie_ids = df_movies_small['movieId'].tolist()
embeddings = df_movies_small['embedding'].tolist()

print("Inserting embeddings into Milvus...")
for i in tqdm(range(0, len(movie_ids), 100)):  # Вставка батчами по 100 записей
    batch_movie_ids = movie_ids[i:i+100]
    batch_embeddings = embeddings[i:i+100]
    collection.insert([batch_movie_ids, batch_embeddings])

# Проверка успешности вставки
print(f"Inserted {collection.num_entities} movie embeddings into Milvus.")

Generating embeddings...


100%|██████████| 800/800 [02:46<00:00,  4.81it/s]
/var/folders/3_/0zlq__wj47xg9llgg6qg6jzc0000gn/T/ipykernel_23298/3182497603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_small['embedding'] = df_movies_small.progress_apply(generate_movie_embeddings, axis=1)


Inserting embeddings into Milvus...


100%|██████████| 8/8 [00:00<00:00, 36.35it/s]

Inserted 0 movie embeddings into Milvus.


In [16]:
query_results = collection.query(
    expr="",
    output_fields=["movie_id", "embedding"],  # поля, которые хотим извлечь
    limit=5  # ограничение на количество возвращаемых объектов
)

for result in query_results:
    print(f"Movie ID: {result['movie_id']}")

Movie ID: 1
Movie ID: 2
Movie ID: 3
Movie ID: 4
Movie ID: 5


In [17]:
# Функция для поиска фильмов по текстовому запросу
def search_movies(query, collection_name="movies_collection", top_k=5):
    # Получение эмбеддинга запроса
    query_embedding = get_embedding(query)
    
    # Подключение к коллекции Milvus
    collection = Collection(collection_name)
    collection.load()  # Загрузка коллекции в память (если не загружена)
    
    # Параметры поиска
    search_params = {
        "metric_type": "IP",  # Тип метрики для расстояния
        "params": {"nprobe": 20}  # Число кластеров для поиска
    }
    
    # Выполнение поиска в Milvus
    results = collection.search(
        data=[query_embedding],  # Векторный запрос
        anns_field="embedding",  # Поле для поиска
        param=search_params,
        limit=top_k,  # Число возвращаемых результатов
        output_fields=["movie_id"]  # Возвращаем только ID фильмов
    )
    
    # Форматирование результатов
    movie_ids = [result.id for result in results[0]]
    distances = [result.distance for result in results[0]]
    return list(zip(movie_ids, distances))  # Возвращаем ID фильмов и их расстояния

# Пример использования функции
query = "dracula"
top_movies = search_movies(query)
print("Top recommended movies:")
for movie in top_movies:
    print(movie, df_movies_small[df_movies_small['movieId'] == movie[0]]['title'])

Top recommended movies:
(12, 330.3630676269531) 11    Dracula: Dead and Loving It (1995)
Name: title, dtype: object
(253, 372.6783752441406) 246    Interview with the Vampire: The Vampire Chroni...
Name: title, dtype: object
(273, 377.08538818359375) 266    Mary Shelley's Frankenstein (Frankenstein) (1994)
Name: title, dtype: object
(653, 378.4429626464844) 633    Dragonheart (1996)
Name: title, dtype: object
(184, 378.5644836425781) 178    Nadja (1994)
Name: title, dtype: object


In [52]:
df_movies_small

,movieId,title,genres,tmdbid,overview,production_countries,runtime,spoken_languages,vote_average,vote_count,embedding
0,1,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",862,"Led by Woody, Andy's toys live happily in his ...",United States of America,81,English,8.0,18253,"[1.2761759, -0.9278991, -0.5428292, -1.0286126..."
1,2,Jumanji (1995),"Adventure,Children,Fantasy",8844,When siblings Judy and Peter discover an encha...,United States of America,104,"English,Français",7.2,10435,"[1.1884694, -0.116759725, -0.6679893, -0.49546..."
2,3,Grumpier Old Men (1995),"Comedy,Romance",15602,A family wedding reignites the ancient feud be...,United States of America,101,English,6.5,374,"[0.919276, -0.29138383, -1.1251851, -1.1663789..."
